<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from torchvision import datasets, transforms
import numpy as np

train_mnist = datasets.MNIST("", train=True, transform=transforms.ToTensor(), download=True)

test_mnist = datasets.MNIST("", train=False, transform=transforms.ToTensor(), download=True)

train = []
test = []
for data in train_mnist:
  train.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])

for data in test_mnist:
  test.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])


print(len(train))
print(len(test))
print(len(train[0][0]), type(train[0][0]))

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw


Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.






60000
10000
784 <class 'numpy.ndarray'>


In [0]:
np.random.seed(1)
w = np.random().random((784, 10))
b = 0.0

def forward(x):
  weighted_sums = []
  for i in range(10):
    weighted_sums.append()

# MSE
def calc_loss(y, y_pred):
  return ((y-y_pred)**2).mean()

# gradient of MSE dJ/dw = 1/N 2x (w*x - y)
def calc_gradient(x, y, y_pred):
  return (np.dot(2*x, y_pred - y)).mean()

print("prediction before training : ", forward(10))

learning_rate = 0.01

for epoch in range(15):
  BATCH_SIZE = 20
  for data in train:
    y_pred = forward(data)

  loss = calc_loss(y, y_pred)

  grad = calc_gradient(x, y, y_pred)

  w -= learning_rate * grad

  print("epoch : ", epoch, "weight : ", w, "loss : ", loss)


[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         